In [1]:
from coursea import *
await init()


2024-01-02 11:14:28,330 I 635507 cfg4py.core:update_config:280 | configuration is
alpha: {data_home: ~/zillionare/alpha/data, tts_server: 'http://127.0.0.1:5002/api/tts?'}
backtest: {url: 'http://192.168.100.114:7080/backtest/api/trade/v0.5/'}
influxdb: {bucket_name: zillionare, enable_compress: true, max_query_size: 5000, org: zillionare,
  token: hwxHycJfp_t6bCOYe2MhEDW4QBOO4FDtgeBWnPR6bGZJGEZ_41m_OHtTJFZKyD2HsbVqkZM8rJNkMvjyoXCG6Q==,
  url: 'http://192.168.100.101:58086'}
notify: {dingtalk_access_token: 58df072143b52368086736cb38236753073ccde6537650cad1d5567747803563,
  keyword: trader}
pluto: {store: ~/zillionare/pluto/store}
redis: {dsn: 'redis://192.168.100.101:56379'}
tasks: {pooling: false, wr: false}

2024-01-02 11:14:28,333 I 635507 /home/aaron/miniconda3/envs/coursea/lib/python3.8/site-packages/omicron/dal/cache.py:init:94 | init redis cache...
2024-01-02 11:14:28,346 I 635507 /home/aaron/miniconda3/envs/coursea/lib/python3.8/site-packages/omicron/dal/cache.py:init:124 | red

init securities done


In [28]:
import pyarrow as pa

code = "000001.XSHE"
bars = await Stock.get_bars(code, 240, FrameType.MIN1)

# 从BarsArray构建table
schema = pa.schema([("frame", pa.date64()),
         ("open", pa.float64()),
         ("high", pa.float64()),
         ("low", pa.float64()),
         ("close", pa.float64()),
         ("volume", pa.float64()),
         ("money", pa.float64()),
         ("factor", pa.float64())
])


table = pa.Table.from_arrays([
    bars[key] for key in bars.dtype.names
], schema=schema)


In [27]:
table


pyarrow.Table
frame: date64[ms]
open: double
high: double
low: double
close: double
volume: double
money: double
factor: double
----
frame: [[2024-01-02,2024-01-02,2024-01-02,2024-01-02,2024-01-02,...,2024-01-02,2024-01-02,2024-01-02,2024-01-02,2024-01-02]]
open: [[9.390000343322754,9.369999885559082,9.380000114440918,9.380000114440918,9.350000381469727,...,9.220000267028809,9.220000267028809,9.210000038146973,9.210000038146973,9.210000038146973]]
high: [[9.420000076293945,9.380000114440918,9.380000114440918,9.380000114440918,9.350000381469727,...,9.220000267028809,9.220000267028809,9.210000038146973,9.210000038146973,9.210000038146973]]
low: [[9.359999656677246,9.369999885559082,9.369999885559082,9.350000381469727,9.34000015258789,...,9.210000038146973,9.210000038146973,9.210000038146973,9.210000038146973,9.210000038146973]]
close: [[9.369999885559082,9.380000114440918,9.369999885559082,9.350000381469727,9.34000015258789,...,9.210000038146973,9.220000267028809,9.210000038146973,9.2100

In [33]:
import pyarrow.parquet as pq

pq.write_table(table, f"/tmp/pyarrow/{code}.parquet")


In [34]:
import pyarrow.dataset as ds

dataset = ds.dataset("/tmp/pyarrow")
dataset


In [37]:
dataset.to_table()


pyarrow.Table
frame: date32[day]
open: double
high: double
low: double
close: double
volume: double
money: double
factor: double
----
frame: [[2024-01-02,2024-01-02,2024-01-02,2024-01-02,2024-01-02,...,2024-01-02,2024-01-02,2024-01-02,2024-01-02,2024-01-02]]
open: [[9.390000343322754,9.369999885559082,9.380000114440918,9.380000114440918,9.350000381469727,...,9.220000267028809,9.220000267028809,9.210000038146973,9.210000038146973,9.210000038146973]]
high: [[9.420000076293945,9.380000114440918,9.380000114440918,9.380000114440918,9.350000381469727,...,9.220000267028809,9.220000267028809,9.210000038146973,9.210000038146973,9.210000038146973]]
low: [[9.359999656677246,9.369999885559082,9.369999885559082,9.350000381469727,9.34000015258789,...,9.210000038146973,9.210000038146973,9.210000038146973,9.210000038146973,9.210000038146973]]
close: [[9.369999885559082,9.380000114440918,9.369999885559082,9.350000381469727,9.34000015258789,...,9.210000038146973,9.220000267028809,9.210000038146973,9.210

In [45]:
import pyarrow as pa

schema = pa.schema([
        ("symbol", pa.string()),
        ("frame", pa.date64()),
        ("open", pa.float32()),
        ("high", pa.float32()),
        ("low", pa.float32()),
        ("close", pa.float32()),
        ("volume", pa.float64()),
        ("money", pa.float64()),
        ("factor", pa.float64())
])


In [64]:

import arrow
import pyarrow.parquet as pq

async def save_1m_bars(codes, dt: datetime.datetime):
    tables = None

    for code in codes:
        bars = await Stock.get_bars(code, 240, FrameType.MIN1, end=dt)
        data = [[code] * len(bars)]
        data.extend([
                    bars[key] for key in bars.dtype.names
                ])
        table = pa.Table.from_arrays(data, schema=schema)
        if tables is None:
            tables = table
        else:
            tables = pa.concat_tables([tables, table])

    # 写入磁盘
    name = arrow.get(dt).format("YYYY-MM-DD")
    pq.write_table(tables, f"/tmp/pyarrow/1m/{name}.parquet")
        
codes = ["000001.XSHE", "600000.XSHG"]
for i in (25, 26, 27, 28, 29):
    dt = datetime.datetime(2023, 12, i, 15)
    await save_1m_bars(codes, dt)


In [ ]:
!tree /tmp/pyarrow


/tmp/pyarrow
├── 1d
├── 1m
│   ├── 2023-12-25.parquet
│   ├── 2023-12-26.parquet
│   ├── 2023-12-27.parquet
│   ├── 2023-12-28.parquet
│   └── 2023-12-29.parquet
└── factors

3 directories, 5 files


In [68]:
# 在分析程序中，加载数据

import pyarrow.dataset as ds

dataset = ds.dataset("/tmp/pyarrow/1m")
dataset.files


['/tmp/pyarrow/1m/2023-12-25.parquet',
 '/tmp/pyarrow/1m/2023-12-26.parquet',
 '/tmp/pyarrow/1m/2023-12-27.parquet',
 '/tmp/pyarrow/1m/2023-12-28.parquet',
 '/tmp/pyarrow/1m/2023-12-29.parquet']

In [83]:
table = dataset.to_table()
table


pyarrow.Table
symbol: string
frame: date32[day]
open: float
high: float
low: float
close: float
volume: double
money: double
factor: double
----
symbol: [["000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE",...,"600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG"],["000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE",...,"600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG"],...,["000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE",...,"600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG"],["000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE","000001.XSHE",...,"600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG","600000.XSHG"]]
frame: [[2023-12-25,2023-12-25,2023-12-25,2023-12-25,2023-12-25,...,2023-12-25,2023-12-25,2023-12-25,2023-12-25,2023-12-25],[2023-12-26,2023-12-26,2023-12-26,2023-12-26,2023-12-26,...,2023-12-26,2023-12-26,2023-12-26,2023-12-26,2023-12-26],...,[2023-12-28,2

如果数据量很大，这会导致内存不够用。因此，很多情况下，我们将使用RecordBatch:


In [84]:
for rb in dataset.to_batches():
    print(rb.to_pandas())


          symbol       frame  open  high   low  close     volume       money  \
0    000001.XSHE  2023-12-25  9.18  9.19  9.15   9.17  1383900.0  12688903.0   
1    000001.XSHE  2023-12-25  9.17  9.17  9.16   9.17   697000.0   6389101.0   
2    000001.XSHE  2023-12-25  9.17  9.18  9.17   9.18   697800.0   6403505.0   
3    000001.XSHE  2023-12-25  9.18  9.18  9.17   9.17   798500.0   7324695.0   
4    000001.XSHE  2023-12-25  9.17  9.18  9.17   9.17   385300.0   3536008.0   
..           ...         ...   ...   ...   ...    ...        ...         ...   
475  600000.XSHG  2023-12-25  6.60  6.60  6.58   6.58    48100.0    317079.0   
476  600000.XSHG  2023-12-25  6.59  6.60  6.58   6.60    55000.0    362527.0   
477  600000.XSHG  2023-12-25  6.60  6.60  6.60   6.60     1100.0      7260.0   
478  600000.XSHG  2023-12-25  6.60  6.60  6.60   6.60        0.0         0.0   
479  600000.XSHG  2023-12-25  6.58  6.58  6.58   6.58   182200.0   1198876.0   

         factor  
0    126.929253  
1  

In [86]:
import pyarrow.compute as pc

filter = (pc.field("frame") > pc.scalar(datetime.datetime(2023, 12, 28, 15)))
dataset.filter(filter).to_table().to_pandas()


,symbol,frame,open,high,low,close,volume,money,factor
0,000001.XSHE,2023-12-29,9.42,9.48,9.41,9.46,3824300.0,36081692.0,126.929253
1,000001.XSHE,2023-12-29,9.46,9.47,9.44,9.46,1002800.0,9481177.0,126.929253
2,000001.XSHE,2023-12-29,9.46,9.46,9.44,9.44,743746.0,7026266.0,126.929253
3,000001.XSHE,2023-12-29,9.44,9.44,9.42,9.43,1062400.0,10023791.0,126.929253
4,000001.XSHE,2023-12-29,9.44,9.44,9.42,9.44,1420600.0,13398608.0,126.929253
...,...,...,...,...,...,...,...,...,...
475,600000.XSHG,2023-12-29,6.62,6.63,6.62,6.62,123600.0,818580.0,15.539983
476,600000.XSHG,2023-12-29,6.63,6.63,6.62,6.62,225500.0,1493633.0,15.539983
477,600000.XSHG,2023-12-29,6.62,6.62,6.62,6.62,0.0,0.0,15.539983
478,600000.XSHG,2023-12-29,6.62,6.62,6.62,6.62,0.0,0.0,15.539983
